In [ ]:
import pandas as pd
wallets_df = pd.read_excel("Wallet id.xlsx")  


In [8]:
wallets = wallets_df['wallet_id'].tolist()
wallets[:5] 


['0x0039f22efb07a647557c7c5d17854cfd6d489ef3',
 '0x06b51c6882b27cb05e712185531c1f74996dd988',
 '0x0795732aacc448030ef374374eaae57d2965c16c',
 '0x0aaa79f1a86bc8136cd0d1ca0d51964f4e3766f9',
 '0x0fe383e5abc200055a7f391f94a5f5d1f844b9ae']

In [ ]:
import numpy as np

def simulate_wallet_activity(wallet_id):
    return {
        "wallet_id": wallet_id,
        "total_borrows": np.random.uniform(0, 10000),
        "total_supplies": np.random.uniform(0, 15000),
        "liquidation_count": np.random.randint(0, 10),
        "health_factor_avg": np.random.uniform(0.5, 3.0),
        "recent_activity_days": np.random.randint(0, 180),
    }


wallet_data = [simulate_wallet_activity(w) for w in wallets]
activity_df = pd.DataFrame(wallet_data)
activity_df.head()


,wallet_id,total_borrows,total_supplies,liquidation_count,health_factor_avg,recent_activity_days
0,0x0039f22efb07a647557c7c5d17854cfd6d489ef3,7471.687267,4665.114586,6,0.825626,34
1,0x06b51c6882b27cb05e712185531c1f74996dd988,6434.972057,12634.327316,4,2.654962,103
2,0x0795732aacc448030ef374374eaae57d2965c16c,6163.365508,3736.770295,5,2.386130,167
3,0x0aaa79f1a86bc8136cd0d1ca0d51964f4e3766f9,38.927563,1244.533714,6,2.076140,147
4,0x0fe383e5abc200055a7f391f94a5f5d1f844b9ae,8468.799670,9841.701744,5,1.817270,176


In [ ]:
from sklearn.preprocessing import MinMaxScaler


activity_df['collateral_ratio'] = activity_df['total_supplies'] / (activity_df['total_borrows'] + 1)

features_to_normalize = ['total_borrows', 'total_supplies', 'liquidation_count', 'health_factor_avg', 'recent_activity_days', 'collateral_ratio']
scaler = MinMaxScaler()
activity_df[features_to_normalize] = scaler.fit_transform(activity_df[features_to_normalize])
activity_df.head()


,wallet_id,total_borrows,total_supplies,liquidation_count,health_factor_avg,recent_activity_days,collateral_ratio
0,0x0039f22efb07a647557c7c5d17854cfd6d489ef3,0.748790,0.309471,0.666667,0.117239,0.177143,0.004113
1,0x06b51c6882b27cb05e712185531c1f74996dd988,0.644349,0.843465,0.444444,0.860404,0.571429,0.013028
2,0x0795732aacc448030ef374374eaae57d2965c16c,0.616987,0.247265,0.555556,0.751191,0.937143,0.003993
3,0x0aaa79f1a86bc8136cd0d1ca0d51964f4e3766f9,0.000000,0.080268,0.666667,0.625258,0.822857,0.207507
4,0x0fe383e5abc200055a7f391f94a5f5d1f844b9ae,0.849241,0.656339,0.555556,0.520093,0.988571,0.007694


In [ ]:
def compute_score(row):
    score = (
        0.2 * (1 - row['liquidation_count']) +
        0.25 * row['health_factor_avg'] +
        0.15 * (1 - row['recent_activity_days']) +
        0.2 * row['collateral_ratio'] +
        0.1 * row['total_supplies'] +
        0.1 * (1 - row['total_borrows'])
    )
    return int(score * 1000)

activity_df['score'] = activity_df.apply(compute_score, axis=1)
activity_df[['wallet_id', 'score']].head()


,wallet_id,score
0,0x0039f22efb07a647557c7c5d17854cfd6d489ef3,276
1,0x06b51c6882b27cb05e712185531c1f74996dd988,513
2,0x0795732aacc448030ef374374eaae57d2965c16c,349
3,0x0aaa79f1a86bc8136cd0d1ca0d51964f4e3766f9,399
4,0x0fe383e5abc200055a7f391f94a5f5d1f844b9ae,302


In [6]:
final_df = activity_df[['wallet_id', 'score']]
final_df.to_csv("wallet_risk_scores.csv", index=False)
print("RRisk scores saved to 'wallet_risk_scores.csv'")


RRisk scores saved to 'wallet_risk_scores.csv'
